In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import adam
from keras.regularizers import l2

from keras.utils import np_utils
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, VGG19, Xception

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

### Data preparation

In [2]:
#os.listdir(train_path)

In [3]:
train_path = os.path.join("dataset", "training")
valid_path = os.path.join("dataset", "validation")
test_path = os.path.join("dataset", "test_new")
classes = ['Apple Red Yellow', 'Apple Golden 1', 'Avocado', 'Avocado ripe', 'Banana',
          'Cocos', 'Dates', 'Granadilla', 'Grape Pink', 'Grape White',
          'Kiwi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes',
          'Nectarine', 'Orange', 'Peach', 'Peach Flat', 'Apricot']
n_classes = len(classes)

In [4]:
img_height, img_width, n_channels = 100, 100, 3

In [5]:
datagen = ImageDataGenerator(rescale=1 / 255)

In [6]:
train_generator =  datagen.flow_from_directory(
    train_path,
    classes=classes,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical')

Found 9719 images belonging to 20 classes.


In [7]:
valid_generator =  datagen.flow_from_directory(
    valid_path,
    classes=classes,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical')

Found 1632 images belonging to 20 classes.


In [8]:
test_generator =  datagen.flow_from_directory(
    test_path,
    classes=classes,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical')

Found 1633 images belonging to 20 classes.


### Model 1. Pretrained VGG16

In [9]:
vgg16_net = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=(img_width, img_height, n_channels))
vgg16_net.trainable = False 

In [10]:
model = Sequential()

model.add(vgg16_net)
model.add(Flatten())

model.add(Dense(256, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation="softmax", kernel_initializer="glorot_uniform", kernel_regularizer=l2(0.01)))

model.compile(loss='categorical_crossentropy',
              optimizer=adam(lr=1e-5), 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1179904   
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
__________

In [13]:
#TensorBoard(log_dir="tb_logs", histogram_freq=1, write_images=True)
callbacks = [EarlyStopping(monitor="val_loss", patience=5),
            ModelCheckpoint("saved_models/model1_vgg16_best1_weights.hdf5", save_best_only=True, monitor="val_loss")]

In [25]:
%%time

history = model.fit_generator(train_generator, 
       epochs=30,                    
       validation_data=valid_generator,
       callbacks=callbacks,
       verbose=1)

Epoch 1/30
304/304 [==============================] - 152s 501ms/step - loss: 6.9979 - acc: 0.6953 - val_loss: 6.4825 - val_acc: 0.8873
Epoch 2/30
304/304 [==============================] - 148s 487ms/step - loss: 6.6229 - acc: 0.7436 - val_loss: 6.1692 - val_acc: 0.9038
Epoch 3/30
304/304 [==============================] - 148s 488ms/step - loss: 6.3013 - acc: 0.7825 - val_loss: 5.8982 - val_acc: 0.9240
Epoch 4/30
304/304 [==============================] - 148s 488ms/step - loss: 6.0242 - acc: 0.8158 - val_loss: 5.6632 - val_acc: 0.9289
Epoch 5/30
304/304 [==============================] - 148s 487ms/step - loss: 5.7696 - acc: 0.8472 - val_loss: 5.4638 - val_acc: 0.9479
Epoch 6/30
304/304 [==============================] - 148s 488ms/step - loss: 5.5657 - acc: 0.8642 - val_loss: 5.2875 - val_acc: 0.9522
Epoch 7/30
304/304 [==============================] - 148s 487ms/step - loss: 5.3648 - acc: 0.8797 - val_loss: 5.0985 - val_acc: 0.9626
Epoch 8/30
304/304 [============================

In [26]:
scores = model.evaluate_generator(test_generator)
print("Test accuracy after last epoch =", scores[1])

Test accuracy after last epoch = 0.9920391916717698


In [27]:
model.save_weights("saved_models/model1_vgg16_best2_weights.hdf5")

In [28]:
# Load best 1 model
model.load_weights("saved_models/model1_vgg16_best1_weights.hdf5")

In [29]:
scores = model.evaluate_generator(test_generator)
print("Test accuracy =", scores[1])

Test accuracy = 0.9920391916717698


In [30]:
scores = model.evaluate_generator(valid_generator)
print("Valid accuracy =", scores[1])

Valid accuracy = 0.991421568627451


In [31]:
scores = model.evaluate_generator(train_generator)
print("Train accuracy =", scores[1])

Train accuracy = 1.0


In [32]:
saved_model = model.to_json()
with open("saved_models/model1_vgg16_architecture.json", "w") as json_file:
    json_file.write(saved_model)